In [1]:
import glob
import os
import json
import re
import logging
import asyncio
from zhihuCrawler import ZhiHuQAPage, AnswerCrawler, QuestionCrawler, PageOperation
def get_query(file_path):
    chinese_pattern = re.compile(r'[\u4e00-\u9fa5]+')
    chinese_matches = chinese_pattern.findall(file_path)
    return chinese_matches[0]


folder_path = "./"
file_lst = glob.glob(os.path.join(folder_path, 'query-*.json'))
data = []


for file_path in file_lst:
    with open(file_path, 'r', encoding='utf-8') as file:
        query = get_query(file_path)
        data.append({query : json.load(file)})
        


In [2]:
logging.basicConfig(
    filename= "./CrawlingAnswers.log", 
    filemode="a+", 
    encoding='utf-8',
    level=logging.INFO,
    format='%(asctime)s %(levelname)s : %(message)s', 
    datefmt="%Y-%m-%d %H:%M:%S",
    )

sum = 0

for query_dict in data:
    query_folder = list(query_dict.keys())[0]
    if os.path.exists("./" + query_folder):
        logging.info(f"当前目录存在文件夹 {query_folder}")
    else:
        os.mkdir("./"+query_folder)
        logging.info(f"创建文件夹 {query_folder}")
    file = open("./" + query_folder + "/问题-序号对照表.txt", "a+", encoding='utf-8')
    for index, q_dict in enumerate(query_dict[query_folder]):
        # 如果当前问题已经进行爬取，则不爬了
        if os.path.exists(f"./{query_folder}/question-{index}-{0}" + ".json") == False:
            # 写入文件序号对应位置
            logging.info(f"开始爬取： {q_dict['question']}")
            file.write(f"question-{index} : {q_dict['question']}\n")
            # 获得 url
            target_url = q_dict['url']
            

            zhihu = ZhiHuQAPage(target_url, proxy_pool=None)

            browser, page = await zhihu.try_visit(headless=False)  # 使用无头模式会跳出验证，不能把网页放到底下？？

            # 进行下拉操作，更新文本，每次下拉50次保存一次文本
            max_down = 10
            for loop in range(int(1000/max_down)):
                # 获取问题描述文本
                question_content = await QuestionCrawler.questionContentCrawl(page)
                # 获取问题标签
                tag_dict = await QuestionCrawler.questionTagCrawl(page=page)
                # 获取关注人数和浏览数
                number_board = await QuestionCrawler.numberBoardCrawl(page=page)
                questionANDAnswers = [{
                    'title' : question_content['title'],
                    'QuestionURL' : target_url,
                    'authorName' : question_content['authorName'],
                    'authorURL' : question_content['authorURL'],
                    'tags' : tag_dict,
                    'content' : question_content['QContentHTML']  
                }]

                isDown = await PageOperation.scrollDown(page=page, max_down=max_down)
                answers_list = await AnswerCrawler.allAnswersTextExtraction(page=page)
                questionANDAnswers.extend(answers_list)
                
                with open(f"./{query_folder}/question-{index}-{loop*max_down}" + ".json", 'w', encoding= 'utf-8') as json_file:
                    json.dump(questionANDAnswers, json_file, ensure_ascii=False, indent=4)
                print(f"question-{index} 写入 {len(questionANDAnswers)} 条数据（问题和答案）")
                logging.info(f"question-{index} 写入 {len(questionANDAnswers)} 条数据（问题和答案）")
                sum += len(questionANDAnswers)
                if isDown:
                    logging.info(f"第 {loop} 次循环完成所有内容加载, 每次循环下拉 {max_down}")
                    break
            await browser.close()  
            await asyncio.sleep(10)
    file.close()
logging.info(f"共写入 {sum} 条数据")

question-3 写入 41 条数据（问题和答案）
question-3 写入 86 条数据（问题和答案）
question-3 写入 111 条数据（问题和答案）
question-3 写入 146 条数据（问题和答案）
question-3 写入 171 条数据（问题和答案）
question-3 写入 196 条数据（问题和答案）
question-3 写入 221 条数据（问题和答案）
question-3 写入 241 条数据（问题和答案）
question-3 写入 266 条数据（问题和答案）
question-3 写入 291 条数据（问题和答案）
question-3 写入 311 条数据（问题和答案）
question-3 写入 325 条数据（问题和答案）
question-4 写入 41 条数据（问题和答案）
question-4 写入 71 条数据（问题和答案）
question-4 写入 121 条数据（问题和答案）
question-4 写入 161 条数据（问题和答案）
question-4 写入 186 条数据（问题和答案）
question-4 写入 216 条数据（问题和答案）
question-4 写入 246 条数据（问题和答案）
question-4 写入 271 条数据（问题和答案）
question-4 写入 306 条数据（问题和答案）
question-4 写入 336 条数据（问题和答案）
question-4 写入 361 条数据（问题和答案）
question-4 写入 386 条数据（问题和答案）
question-4 写入 406 条数据（问题和答案）
question-4 写入 431 条数据（问题和答案）
question-4 写入 456 条数据（问题和答案）
question-4 写入 476 条数据（问题和答案）
question-4 写入 501 条数据（问题和答案）
question-4 写入 526 条数据（问题和答案）
question-4 写入 551 条数据（问题和答案）
question-4 写入 591 条数据（问题和答案）
question-4 写入 611 条数据（问题和答案）
question-4 写入 631 条数据（问题和答案）
question-4 写入 648 